In [1]:
# generate links / json states programatically by cell type or anything, probably hack-y but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np
import json

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
# timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [93]:
def make_json(seg_ids, hidden_ids):
    state = {
    "layers": [
      {
        "source": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/em/rechunked",
        "type": "image",
        "blend": "default",
        "shaderControls": {},
        "name": "FANC EM"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/meshes/FANC/FANC_neurons/meshes",
        "selectedAlpha": 0.4,
        "colorSeed": 3788587020,
        "segmentColors": {
        "648518346494688675": "#fbdb03",
        "648518346481319311": "#fbdb03",
        "648518346489942455": "#fbdb03"
      },
        "segments": seg_ids,
        "hiddenSegments": hidden_ids,
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "published FANC neurons"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
        "objectAlpha": 0.1,
        "hideSegmentZero": False,
        "ignoreSegmentInteractions": True,
        "segmentColors": {
          "1": "#bfbfbf",
          "2": "#d343d6"
        },
        "segments": [
          "1",
          "2"
        ],
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "region outlines"
      }
    ],
    "navigation": {
      "pose": {
        "position": {
          "voxelSize": [
            4.300000190734863,
            4.300000190734863,
            45
          ],
          "voxelCoordinates": [
          36053,
          126116,
          455
        ]
        }
      },
      "zoomFactor": 12
    },
    "showAxisLines": False,
    "perspectiveOrientation": [
    0,
    1,
    0,
    0
  ],
    "showDefaultAnnotations": False,
    "perspectiveZoom": 4495.05827428684,
    "showSlices": False,
    "gpuMemoryLimit": 4000000000,
    "systemMemoryLimit": 4000000000,
    "concurrentDownloads": 64,
    "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post",
    "selectedLayer": {
      "layer": "published FANC neurons",
      "visible": True
    },
      "layout": {
      "type": "3d",
      "orthographicProjection": True
      }
    }
    return state

In [5]:
# get left wing MNs from cave annotation table
wing_table = client.materialize.query_table('wing_motor_neuron_table_v0')

wing_table['side'] = [x.strip()[-1] for x in wing_table.classification_system]
wingL_table = wing_table[wing_table.side.isin(['L'])]

wing_MN_table = wingL_table[~wingL_table.cell_type.isin(['nonMN_unk3','nonMN_unk2','nonMN_unk1','nonMN_PSI','tt','ttb_u','ttc_u', 'PSn_u'])] # other efferent neurons
wing_MN_table.shape # should be 29

(29, 10)

In [6]:
# get left leg MNs from cave annotation table
leg_table = client.materialize.query_table('motor_neuron_table_v7')

leg_table['side'] = [x.strip()[-1] for x in leg_table.classification_system]
leg_MN_table = leg_table[leg_table.side.isin(['L'])]

leg_MN_table.shape # should be 69

(69, 10)

In [7]:
leg_dict = {
    "coxa_swing" : [648518346491659326,648518346479325585,648518346517827432,648518346486895331,648518346487504531,648518346517437482,648518346487756866],
    "coxa_stance" : [648518346478992848,648518346488854257,648518346490902957,648518346489802071,648518346479077839,648518346489076201],
    "trochanter_extend" : [648518346504856435,648518346496195084,648518346482347156,648518346492578233,648518346515708666,648518346467061998,648518346502537286,648518346498089882],
    "trochanter_flex" : [648518346487577002,648518346514275550,648518346517824616,648518346496812333,648518346479840662,648518346499918803,648518346514277931,648518346488873565,648518346498498324,648518346490899373,648518346489470041],
    "femur_reductor" : [648518346487743188,648518346493238592,648518346496372087,648518346502536518,648518346492338559,648518346494195703],
    "tibia_extend" : [648518346493238080,648518346495797355],
    "tibia_flex_A" : [648518346496059082,648518346494196727,648518346496932836,648518346484809885,648518346493836024,648518346486906595,648518346482926501,648518346489731855,648518346494269831,648518346499176513],
    "tibia_flex_B" : [648518346490901677,648518346496057290,648518346493202674,648518346479331004,648518346482925989,648518346517830760,648518346495467855],
    "tibia_flex_C" : [648518346496852276,648518346491041571],
    "substrate_grip" : [648518346491158561,648518346475434081,648518346504867443,648518346486902499,648518346484620291,648518346515711482,648518346493203442,648518346496934372],
    "tarsus_control" : [648518346486890393,648518346491158817]
}

wing_dict = {
    "DLM" : [648518346472269618,648518346486563887,648518346515087943,648518346494250631,648518346486629017],
    "DVM" : [648518346492294527,648518346503730081,648518346488827114,648518346512153646,648518346492384195,648518346499049025,648518346509427075],
    "tension" : [648518346492244419,648518346497125761,648518346490085915,648518346486682876,648518346472953700],
    "steerA" : [648518346488622926,648518346487480490,648518346509604297],
    "steerB" : [648518346470491902,648518346471876251,648518346499256923],
    "steerC" : [648518346493634040,648518346501373528],
    "steerhg2" : [648518346492048172],
    "steerD" : [648518346482131604,648518346480882144,648518346515574010,648518346492294783]
}

In [96]:
# MN table with links

json_base = 'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/'
show_ids = [648518346494688675,648518346489931738,648518346500087558,648518346514282206,648518346481319311,648518346489942455]
hidden_ids = []
title = 'gfc2_tt'

formatted_ids = [str(x) for x in show_ids]
state = make_json(seg_ids = formatted_ids,
                        hidden_ids = hidden_ids)
json_title = str(title)

# save a json
with open('jsons/{}.json'.format(json_title), "w") as outfile:
        json.dump(state, outfile)

table_link = json_base + json_title
hyperlink = 'HYPERLINK("{}.json";"{}")'.format(table_link, json_title)


In [97]:
hyperlink

'HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/gfc2_tension.json";"gfc2_tension")'

In [60]:
table_link

'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/feti_seti'

In [66]:
wingL_table[wingL_table.cell_type.isin(['nonMN_unk3','nonMN_unk2','nonMN_unk1','nonMN_PSI','tt','ttb_u','ttc_u', 'PSn_u'])]

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position,side
21,26,2022-11-30 17:07:21.605759+00:00,NaN,t,mesoAN_L,PSn_u,72907242990797785,648518346492244419,"[26432, 156112, 2594]",L
26,31,2022-11-30 17:07:21.609525+00:00,NaN,t,PDMN_L,tt,72343330830526265,648518346489942455,"[9072, 128072, 2373]",L
27,32,2022-11-30 17:07:21.610286+00:00,NaN,t,PDMN_L,ttb_u,72343330427863868,648518346494688675,"[8464, 128664, 1591]",L
28,33,2022-11-30 17:07:21.611046+00:00,NaN,t,PDMN_L,ttc_u,72343605305769879,648518346481319311,"[8344, 135464, 1599]",L
29,34,2022-11-30 17:07:21.611784+00:00,NaN,t,PDMN_L,nonMN_PSI,74172918112005839,648518346486137513,"[62584, 128136, 2213]",L
30,35,2022-11-30 17:07:21.612814+00:00,NaN,t,PDMN_L,nonMN_unk1,72413355977321122,648518346472328139,"[10492, 118012, 2403]",L
31,36,2022-11-30 17:07:21.613611+00:00,NaN,t,PDMN_L,nonMN_unk2,73328975023585836,648518346514279390,"[38776, 142936, 3792]",L
32,37,2022-11-30 17:07:21.614381+00:00,NaN,t,ADMN_L,nonMN_unk3,72483724453108761,648518346483195582,"[13824, 116860, 1883]",L


In [ ]:
648518346486137513,648518346492244419,648518346472328139,648518346514279390,648518346483195582